In [2]:
from functions_collection_copy import *
import random as random
import Bio
import numpy as np
import sys
import os
import timeit
from IPython.display import clear_output

score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5
seqs=['aaatt',
 'cagtcgtt',
 'agctg',
 'ctgtagaac',
 'gcagg',
 'tttttt',
 'agcggcg',
 'cacccgtga',
 'tctgcag',
 'taactcata']

In [98]:
def new_assembly_Gus_with_inc_ordering(seqs, score_matrix, gap_cost, return_center_string=False, check_integrity=False):
    # STEP 1: Find the center string, s1
    matrix = np.full((len(seqs), len(seqs)), np.nan)
    # loop over all distinct pairs
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
            matrix[i, j] = get_cost_2(linear_C(gap_cost, score_matrix, seq1, seq2))

    # find the center string/guide 
    s1_idx = int(np.argmin(matrix.sum(axis=1)))
    print(matrix.sum(axis=1))
    print(seqs[s1_idx])
    s1 = seqs[s1_idx]
    new_order={}
    for i,val in enumerate(matrix[s1_idx]):
        print(val)
        print(i)
        new_order[i]=int(val)
    new_order=dict(sorted(new_order.items(), key=lambda item: item[1]))
    for index, key in enumerate(new_order):
        new_order[key] = index
    new_seqs=[None for _ in range(len(seqs))]
    for key,value in new_order.items():
        new_seqs[value]=seqs[key]
    seqs=new_seqs
    # STEP 2: Construct alignment M
    M: list[list[str]] = [[letter] for letter in [*s1]]
    cost_list = []
    for i in range(1, len(seqs)):
        cost = linear_C(gap_cost, score_matrix, s1, seqs[i])
        cost_list.append(get_cost_2(cost))

        # prepare A-matrix for extension
        alignment1_str, alignment2_str = linear_backtrack(s1, seqs[i], cost, score_matrix, gap_cost)
        alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
        A = [list(e) for e in zip(alignment1, alignment2)]

        # extend
        Mk = extend_alignment(M, A)
        M = Mk

    # ACTUALLY COMPUTE (approximate) COST
    total_cost = compute_cost(M, score_matrix, gap_cost)
    if check_integrity == True:
        integrity_check_Gus(seqs, M, score_matrix,gap_cost,matrix,s1_idx)
    return matrix, M, total_cost

In [101]:
seqs=['aaatttttttt',
 'cagtcgtttttttttt',
 'agctgttttttttt',
 'ctgtagaac',
 'gcagg',
 'tttttt',
 'agcggcg',
 'cacccgtga',
 'tctgcag',
 'taactcata']
res_inc_gus=new_assembly_Gus_with_inc_ordering(seqs,score_matrix,gap_cost,check_integrity=True)[1]
res_old_gus=new_assembly_Gus_x(seqs,score_matrix,gap_cost,check_integrity=True)[1]

print(res_inc_gus)
print(res_old_gus)

[307. 437. 368. 306. 304. 271. 277. 276. 275. 271.]
tttttt
25.0
0
50.0
1
40.0
2
29.0
3
27.0
4
0.0
5
29.0
6
28.0
7
19.0
8
24.0
9
Im just gonna start this int check by printint the strings/sequences dammit: ['tttttt', 'tctgcag', 'taactcata', 'aaatttttttt', 'gcagg', 'cacccgtga', 'ctgtagaac', 'agcggcg', 'agctgttttttttt', 'cagtcgtttttttttt']
seq and new_str_no_gaps were: tttttt, tttttt
integrity check 1 passed for seq 0
seq and new_str_no_gaps were: tctgcag, tctgcag
integrity check 1 passed for seq 1
seq and new_str_no_gaps were: taactcata, taactcata
integrity check 1 passed for seq 2
seq and new_str_no_gaps were: aaatttttttt, aaatttttttt
integrity check 1 passed for seq 3
seq and new_str_no_gaps were: gcagg, gcagg
integrity check 1 passed for seq 4
seq and new_str_no_gaps were: cacccgtga, cacccgtga
integrity check 1 passed for seq 5
seq and new_str_no_gaps were: ctgtagaac, ctgtagaac
integrity check 1 passed for seq 6
seq and new_str_no_gaps were: agcggcg, agcggcg
integrity check 1 passed f